<a href="https://colab.research.google.com/github/YassGan/ON_The_Fly_Gaussian_Splatting/blob/main/ON_The_FLy_GS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making sure that we are using a GPU

In [1]:
!nvidia-smi

Wed Apr 23 16:00:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Python downgrading


In [2]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y python=3.7
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')
!python --version  # Should say Python 3.7.x

--2025-04-23 16:00:18--  https://repo.anaconda.com/miniconda/Miniconda3-py37_23.1.0-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90665082 (86M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  86.46M   259MB/s    in 0.3s    

2025-04-23 16:00:19 (259 MB/s) - ‘mini.sh’ saved [90665082/90665082]

PREFIX=/usr/local
Unpacking payload ...
                                                                               
Installing base environment...





Preparing transaction: - \ | / done
Executing transaction: \ | / - \ | / - \ | / - \ | / - \ | / done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running t

# CUDA 11.8

In [3]:
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit --no-drm --no-man-page
import os
os.environ['PATH'] += ':/usr/local/cuda-11.8/bin'
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda-11.8/lib64:/usr/lib64-nvidia'
!nvcc --version  # Should show CUDA 11.8

--2025-04-23 16:00:43--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 2.16.106.132, 2.16.106.152
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|2.16.106.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   123MB/s    in 19s     

2025-04-23 16:01:02 (221 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


#Pytorch with cuda

In [4]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 GB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 70.6 MB/s eta 0:00:00


# Verification of the versions

In [5]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.version.cuda)        # Should be 11.3 (from PyTorch)
print(torch.cuda.get_device_name(0))  # Should show GPU

True
12.4
Tesla T4


In [6]:
##making sure that we are always using the GPU and not a CPU
!nvidia-smi

Wed Apr 23 16:07:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              8W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
!pip install pytorch3d


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61406 sha256=9d3c494775743b3435354655f74153e14d0f9efbde040f50b18c24326bfc9ee6
  Stored in directory: /root/.cache/pip/wheels/12/a2/36/21b9bde5f8deeeb6312efe88ddde26a51facbd2089f32917b3
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=4cea9db18402ee193e29149b99380fc47fbdfffcad99a198ae28dab037542106
  Stored in directory: /root/.cache/pip/wheels/96/9a/78/61eeeec98da40f44085da9ba3fec952b4ab7224f5c5be75126
Successfully built fvcore iopath


In [9]:
!pip install open3d


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.4/420.4 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
%cd /content
!git clone --recursive https://github.com/xywjohn/GS_On-The-Fly
!pip install -q plyfile



/content
Cloning into 'GS_On-The-Fly'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 266 (from 1)
Receiving objects: 100% (269/269), 52.60 MiB | 38.34 MiB/s, done.
Resolving deltas: 100% (94/94), done.


In [12]:
!unzip /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/diff-gaussian-rasterization.zip -d /content/GS_On-The-Fly/submodules/


Archive:  /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/diff-gaussian-rasterization.zip
  inflating: /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/.gitignore  
  inflating: /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/.gitmodules  
   creating: /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/build/
   creating: /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/build/bdist.linux-x86_64/
   creating: /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/build/lib.linux-x86_64-cpython-37/
   creating: /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/build/lib.linux-x86_64-cpython-37/diff_gaussian_rasterization/
  inflating: /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/build/lib.linux-x86_64-cpython-37/diff_gaussian_rasterization/_C.cpython-37m-x86_64-linux-gnu.so  
  inflating: /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization/build/lib.linux-x86_64-cpython-37/diff_

In [13]:
%cd /content/GS_On-The-Fly
!pip install -q /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization
!pip install -q /content/GS_On-The-Fly/submodules/simple-knn

/content/GS_On-The-Fly
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [31]:
!pip install -q /content/GS_On-The-Fly/submodules/fused-ssim

  Preparing metadata (setup.py) ... done


In [16]:
# Verify GS_On-The-Fly submodules
!ls -la /content/GS_On-The-Fly/submodules/diff-gaussian-rasterization
!ls -la /content/GS_On-The-Fly/submodules/simple-knn

total 12712
drwxr-xr-x 7 root root     4096 Apr 23 16:13 .
drwxr-xr-x 5 root root     4096 Apr 23 16:11 ..
-rw-r--r-- 1 root root        1 Apr 23 16:11 111
drwxrwxrwx 5 root root     4096 Apr 22 07:51 build
-rw-rw-rw- 1 root root     1046 Apr 22 07:51 CMakeLists.txt
-rw-rw-rw- 1 root root    49588 Apr 22 07:51 conv.cu
drwxrwxrwx 2 root root     4096 Apr 22 07:51 cuda_rasterizer
drwxrwxrwx 3 root root     4096 Apr 22 07:51 diff_gaussian_rasterization
drwxrwxrwx 2 root root     4096 Apr 23 16:18 diff_gaussian_rasterization.egg-info
-rw-r--r-- 1 root root 12881895 Apr 23 16:11 diff-gaussian-rasterization.zip
-rw-rw-rw- 1 root root      693 Apr 22 07:51 ext.cpp
-rw-rw-rw- 1 root root       51 Apr 22 07:51 .gitignore
-rw-rw-rw- 1 root root       95 Apr 22 07:51 .gitmodules
-rw-rw-rw- 1 root root     4275 Apr 22 07:51 LICENSE.md
-rw-rw-rw- 1 root root     9699 Apr 22 07:51 rasterize_points.cu
-rw-rw-rw- 1 root root     2894 Apr 22 07:51 rasterize_points.h
-rw-rw-rw- 1 root root      871 Apr 

In [20]:
!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/1X3KiQR_bva6nUXQEznZqgHbkam91kCVB -O /content/GS_On-The-Fly

Retrieving folder contents
Retrieving folder 13Qsj6ujVnv_DM81sM71IacVVS6EiwEhS demo_data
Retrieving folder 1Z7e4Fno7QDSWLP3jCtsPk-oE9QKkbCTz data1
Retrieving folder 1QRiRigIkREX374xO0BTxUBHUIwR3OsUG images
Processing file 1JesLwjhte6IUJvax95QjGvLRVFrJxEOg 10002.jpg
Processing file 16Gx1qTfXemC6eLkyw9RH3AJv7dpz5zw7 10005.jpg
Processing file 1nj3hYJjB3oPG8DDEPO5jKdz4vbtnYy1B 10008.jpg
Processing file 1GXt9IcIeGsykzl4PR3Z8YjTh77tBg1xr 10011.jpg
Processing file 1Mwb_-1yerxrH0nGCdVWjwYzQT8woJ5Vd 10014.jpg
Processing file 1PEvyWRvjG2CPXnziiG8sqirwvfiktv4Y 10017.jpg
Processing file 1tm4Jg2Fnssd8TAbUxmK95XQJhXjsAFGk 10020.jpg
Processing file 1Fz01DNkTB8_birXUZ1o4f5pHAQ5vYc2p 10023.jpg
Processing file 13EhMpVOb_EzZASAwL-8M3cdDXrLIeL1d 10026.jpg
Processing file 1lCdpXM2trl6_52gFFm4CloaNgw7Gp4tT 10029.jpg
Processing file 11N6UQDYjwk8JtcJAv05bkvzRx8vk_z3a 10032.jpg
Processing file 1Wp7QKPl-M-G0Gwe4GDQ1OILSCUE93r7J 10035.jpg
Processing file 1uIMs1lbvG9AyRHe_TCnmTA1r0H6HGz_N 10038.jpg
Processing fil

In [22]:
# Check GS_On-The-Fly directory
!ls -la /content/GS_On-The-Fly
# Check for demo_data and On-The-Fly folders
!ls -la /content/GS_On-The-Fly/demo_data
!ls -la /content/GS_On-The-Fly/demo_data/data1
!ls -la /content/GS_On-The-Fly/On-The-Fly
!ls -la /content/GS_On-The-Fly/On-The-Fly/data1

total 24792
drwxr-xr-x 12 root root     4096 Apr 23 16:11 .
drwxr-xr-x  1 root root     4096 Apr 23 16:18 ..
-rw-r--r--  1 root root   788504 Apr 23 16:11 2.png
drwxr-xr-x  3 root root     4096 Apr 23 16:11 arguments
drwxr-xr-x  2 root root     4096 Apr 23 16:11 assets
-rw-r--r--  1 root root    49329 Apr 23 16:11 ContinuosProgressiveTrain.py
-rw-r--r--  1 root root     5349 Apr 23 16:11 convert.py
-rw-r--r--  1 root root     2486 Apr 23 16:11 DatasetPrepare.py
drwxr-xr-x  3 root root     4096 Apr 23 16:11 demo_data
-rw-r--r--  1 root root      334 Apr 23 16:11 environment.yml
-rw-r--r--  1 root root 24451473 Apr 23 16:11 Final_Demo.mp4
-rw-r--r--  1 root root     3340 Apr 23 16:11 full_eval.py
drwxr-xr-x  3 root root     4096 Apr 23 16:11 gaussian_renderer
drwxr-xr-x  8 root root     4096 Apr 23 16:11 .git
drwxr-xr-x  3 root root     4096 Apr 23 16:11 lpipsPyTorch
-rw-r--r--  1 root root     4143 Apr 23 16:11 metrics.py
drwxr-xr-x  3 root root     4096 Apr 23 16:11 On-The-Fly
-rw-r--r

# Mounting the drive

In [26]:


import os
import zipfile
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
# Paths




Mounted at /content/drive


In [15]:
!pip list

Package                     Version
--------------------------- ------------------
addict                      2.4.0
attrs                       24.2.0
backcall                    0.2.0
brotlipy                    0.7.0
certifi                     2022.12.7
cffi                        1.15.1
charset-normalizer          2.0.4
click                       8.1.8
comm                        0.1.4
conda                       23.1.0
conda-content-trust         0.1.3
conda-package-handling      2.0.2
conda_package_streaming     0.7.0
ConfigArgParse              1.7
cryptography                38.0.4
cycler                      0.11.0
dash                        2.15.0
dash-core-components        2.0.0
dash-html-components        2.0.0
dash-table                  5.0.0
decorator                   5.1.1
diff-gaussian-rasterization 0.0.0
fastjsonschema              2.21.1
Flask                       2.2.5
flit_core                   3.6.0
fonttools                   4.38.0
fvcore                 

In [27]:
# Create data directory
!mkdir -p /content/GS_On-The-Fly/data

# Unzip the files
!unzip /content/drive/MyDrive/demo_data-20250423T164654Z-001.zip -d /content/GS_On-The-Fly/data
!unzip /content/drive/MyDrive/On-The-Fly-20250423T164703Z-001.zip -d /content/GS_On-The-Fly/data

# Verify the structure
!ls -la /content/GS_On-The-Fly/data
!ls -la /content/GS_On-The-Fly/data/demo_data/data1
!ls -la /content/GS_On-The-Fly/data/On-The-Fly/data1

Archive:  /content/drive/MyDrive/demo_data-20250423T164654Z-001.zip
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10050.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10119.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10011.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10074.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10041.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/sparse/0/cameras.bin  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10077.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10116.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10086.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10047.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10071.jpg  
  inflating: /content/GS_On-The-Fly/data/demo_data/data1/images/10107.jpg  
  inflating: /co

In [28]:
# Configure DatasetPrepare.py
!sed -i 's|DatasetName = .*|DatasetName = ["data1"]|' /content/GS_On-The-Fly/DatasetPrepare.py
!sed -i 's|SourceImageDataset = .*|SourceImageDataset = "/content/GS_On-The-Fly/data/demo_data"|' /content/GS_On-The-Fly/DatasetPrepare.py
!sed -i 's|On_The_Fly_Dataset = .*|On_The_Fly_Dataset = "/content/GS_On-The-Fly/data/On-The-Fly"|' /content/GS_On-The-Fly/DatasetPrepare.py
# Verify configuration
!cat /content/GS_On-The-Fly/DatasetPrepare.py | grep -E 'DatasetName|SourceImageDataset|On_The_Fly_Dataset'
# Run DatasetPrepare.py
%cd /content/GS_On-The-Fly
!/usr/local/bin/python3.7 DatasetPrepare.py

DatasetName = ["data1"]
SourceImageDataset = "/content/GS_On-The-Fly/data/demo_data"
On_The_Fly_Dataset = "/content/GS_On-The-Fly/data/On-The-Fly"
for dataset in DatasetName:
    SourceImagesDir = os.path.join(SourceImageDataset, r"{}/images".format(dataset))
    MainDir = os.path.join(On_The_Fly_Dataset, dataset)
/content/GS_On-The-Fly
Resize progress data1: 100% 117/117 [00:00<00:00, 4880.54it/s]
/content/GS_On-The-Fly/data/On-The-Fly/data1
Copy progress data1: 100% 103/103 [00:14<00:00,  7.21it/s]


In [38]:
%cd /content/GS_On-The-Fly
!python ContinuosProgressiveTrain.py \
  --Source_Path_Dir /content/GS_On-The-Fly/data/On-The-Fly/data1 \
  --Model_Path_Dir /content/GS_On-The-Fly/OutputDir \
  --FinalOptimizationIterations 4000

/content/GS_On-The-Fly
Optimizing /content/GS_On-The-Fly/data/On-The-Fly/data1
[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118] [23/04 17:52:59]
Mean: 195, MergeScene_Densification_Interval: 100, OpacityThreshold: 0.005, InitialTrainingTimesSetZero: False, UseDifferentImageLr: False, UseDepthLoss: False, UseScaleLoss: False, UseNormalLoss: False [23/04 17:52:59]
Output folder: /content/GS_On-The-Fly/OutputDir/16 [23/04 17:52:59]
Tensorboard not available: not logging progress [23/04 17:52:59]
Reading camera 16/16 [23/04 17:52:59]
Loading Training Cameras [23/04 17:52:59]
Loading Test Cameras [23/04 17:53:02]
N

In [33]:
!ls -l /content/GS_On-The-Fly/data/On-The-Fly/data1/16/images/ | wc -l
!ls -l /content/GS_On-The-Fly/data/On-The-Fly/data1/16/sparse/0/

17
total 3172
-rw-r--r-- 1 root root      64 Apr 15 09:01 cameras.bin
-rw-r--r-- 1 root root    1114 Apr 15 09:01 imageMatchMatrix.txt
-rw-r--r-- 1 root root 1950888 Apr 15 09:01 images.bin
-rw-r--r-- 1 root root      97 Apr 15 09:01 imagesNames.txt
-rw-r--r-- 1 root root  994062 Apr 15 09:01 points3D.bin
-rw-r--r-- 1 root root  283031 Apr 15 09:01 points3D.ply
